In [ ]:
from __future__ import print_function
import sys
sys.path.insert(0, '..')

import pandas as pd
#from pathlib import Path
import torch
from torch import nn
from torch.optim import Adam

from torch.utils.data import random_split
from sound_dataset import SoundDS

import datetime
from early_stopping import EarlyStopping

from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200


import librosa
import soundfile as sf

from sound_utility import AudioUtil

from sklearn.manifold import TSNE

from torch.nn import functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from tqdm import tqdm

import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn. functional as F
import torch.optim as optim
import os
import sys


In [ ]:
batch_size = 1
learning_rate = 0.0001
max_epoch = 100
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available:
    device = torch.device("cuda")    
else:
    device = torch.device("cpu")
num_workers = 5
load_epoch = -1
generate = True

class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_size=50,num_classes=4):
        super(Model,self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_size = latent_size
        self.num_classes = num_classes
        #(16x161318 and 646694x400)
        # For encode
        self.conv1 = nn.Conv2d(1, 8, kernel_size=5)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)
        #self.linear1 = nn.Linear(4*4*32,300)
        #self.mu = nn.Linear(300, self.latent_size)
        #self.logvar = nn.Linear(300, self.latent_size)
        #self.linear1 = nn.Linear(self.input_dim,self.hidden_dim)
        self.linear1 = nn.Linear(646694,hidden_dim)
        #self.linear1 = nn.Linear(161318,hidden_dim)
        
        self.mu = nn.Linear(self.hidden_dim, self.latent_size)
        
        self.logvar = nn.Linear(self.hidden_dim, self.latent_size)
        
        # For decoder
        #self.linear2 = nn.Linear(self.latent_size + self.num_classes, 300)
        #self.linear3 = nn.Linear(300,4*4*32)
        self.linear2 = nn.Linear(self.latent_size + self.num_classes, self.hidden_dim)
        self.linear3 = nn.Linear(self.hidden_dim, self.input_dim)        
        self.conv3 = nn.ConvTranspose2d(16, 8, kernel_size=5)
        self.conv4 = nn.ConvTranspose2d(8, 1, kernel_size=5, padding=4)
        #self.conv5 = nn.ConvTranspose2d(1, 1, kernel_size=5, stride=2, padding=1)


    def encode(self,x,y):
        y = torch.argmax(y, dim=1).reshape((y.shape[0],1,1))
        y = torch.ones(x.shape).to(device)*y
        t = torch.cat((x,y),dim=1)        
        t = F.relu(self.conv1(t))        
        t = F.relu(self.conv2(t))
        t = self.Flatten(t)
        t = F.relu(self.linear1(t))
        mu = self.mu(t)
        logvar = self.logvar(t)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
        
        #std = torch.exp(logvar) # made a mistake removing 0.5
        #epsilon = torch.rand_like(std)
        #return mu + std * epsilon
            
    def Flatten(self,x):
        #print('x shape:',x.shape)
        batch_size = x.size()[0]
        return x.view(batch_size, -1)
    
    def unFlatten(self, x):
        #print('flat x:',x.shape)
        batch_size = x.size()[0]
        #print('batch_size:',batch_size)
        #return x.view(1, batch_size, 4108, 98)
        #4097, 87
        #return x.reshape((batch_size, 4097, 87))
        return x.reshape((batch_size, 4097, 87))
        #return x.view(1, batch_size, 4097, 87)

    def decode(self, z,y):

        #print('z shape:', z.shape)
        #print('y shape:', y.shape)
            
        y_reshaped = y.unsqueeze(0).expand(z.shape[0], -1)
        inputs = torch.cat([z, y_reshaped], 1)

        t = F.relu(self.linear2(inputs))
        t = F.relu(self.linear3(t))
        #print('linear3:',t.shape)
        t = self.unFlatten(t)
        t = F.relu(self.conv3(t))
        #print('t conv3:', t.shape)
        t = F.relu(self.conv4(t))
        #print('t conv4:', t.shape)
        #t = F.relu(self.conv5(t))
        #print('t conv5:', t.shape)
        #t = F.tanh(self.conv5(t))
        return F.tanh(t)


    def forward(self, x, y):
        
        mu, logvar = self.encode(x,y)
        z = self.reparameterize(mu,logvar)

        # Class conditioning
        #z = torch.cat((z, y.float()), dim=1)
        y=y[0]
        
        #print('inputs:',inputs.shape)
        
        pred = self.decode(z,y)
        #print('pred:',pred.shape)
        return pred, mu, logvar


def one_hot(labels, class_size):
    targets = torch.zeros(labels.size(0), class_size)
    for i, label in enumerate(labels):
        targets[i, label] = 1
    return targets.to(device)

def stft_for_reconstruction(x, fft_size, hopsamp):
    window = np.hanning(fft_size)
    fft_size = int(fft_size)
    hopsamp = int(hopsamp)
    return np.array([np.fft.rfft(window*x[i:i+fft_size])
                     for i in range(0, len(x)-fft_size, hopsamp)])

def istft_for_reconstruction(X, fft_size, hopsamp):
    fft_size = int(fft_size)
    hopsamp = int(hopsamp)
    window = np.hanning(fft_size)
    time_slices = X.shape[0]
    len_samples = int(time_slices*hopsamp + fft_size)
    x = np.zeros(len_samples)
    for n,i in enumerate(range(0, len(x)-fft_size, hopsamp)):
        x[i:i+fft_size] += window*np.real(np.fft.irfft(X[n]))
    return x

def reconstruct_signal_griffin_lim(magnitude_spectrogram, fft_size, hopsamp, iterations):
    time_slices = magnitude_spectrogram.shape[0]
    len_samples = int(time_slices*hopsamp + fft_size)
    # Initialize the reconstructed signal to noise.
    x_reconstruct = np.random.randn(len_samples)
    n = iterations # number of iterations of Griffin-Lim algorithm.
    rmse = []
    while n > 0:
        n -= 1
        reconstruction_spectrogram = stft_for_reconstruction(x_reconstruct, fft_size, hopsamp)
        reconstruction_angle = np.angle(reconstruction_spectrogram)
        # Discard magnitude part of the reconstruction and use the supplied magnitude spectrogram instead.
        proposal_spectrogram = magnitude_spectrogram*np.exp(1.0j*reconstruction_angle)
        prev_x = x_reconstruct
        x_reconstruct = istft_for_reconstruction(proposal_spectrogram, fft_size, hopsamp)
        diff = np.sqrt(sum((x_reconstruct - prev_x)**2)/x_reconstruct.size)
        rmse.append(diff)
    return x_reconstruct, rmse


def loss_function(x, pred, mu, logvar):
    recon_loss = F.mse_loss(pred, x, reduction='sum')
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    #return recon_loss, kld
    return recon_loss + kld

def train(epoch, model, train_loader, optim):
    reconstruction_loss = 0
    kld_loss = 0
    total_loss = 0
    class_size = 4
    #for i,(x,y) in enumerate(train_loader):
    for batch_idx, (spectrogram,label) in enumerate(train_loader):
        try:
            #label = np.zeros((x.shape[0], 10))
            #label[np.arange(x.shape[0]), y] = 1
            #label = torch.tensor(label)

            spectrogram_tensor = spectrogram.to(dtype=torch.float32)

            label = label.to(device)
            data = spectrogram_tensor.to(device)
            label = one_hot(label, class_size)

            optim.zero_grad()   
            pred, mu, logvar = model(data.to(device),label.to(device))
            
            recon_loss, kld = loss_function(data.to(device),pred, mu, logvar)
            loss = recon_loss + kld
            loss.backward()
            optim.step()

            total_loss += loss.cpu().data.numpy()*data.shape[0]
            reconstruction_loss += recon_loss.cpu().data.numpy()*data.shape[0]
            kld_loss += kld.cpu().data.numpy()*data.shape[0]
            if i == 0:
                print("Gradients")
                for name,param in model.named_parameters():
                    if "bias" in name:
                        print(name,param.grad[0],end=" ")
                    else:
                        print(name,param.grad[0,0],end=" ")
                    print()
        except Exception as e:
            #traceback.print_exe()
            torch.cuda.empty_cache()
            continue
    
    reconstruction_loss /= len(train_loader.dataset)
    kld_loss /= len(train_loader.dataset)
    total_loss /= len(train_loader.dataset)
    return total_loss, kld_loss,reconstruction_loss

def test(epoch, model, test_loader):
    reconstruction_loss = 0
    kld_loss = 0
    total_loss = 0
    with torch.no_grad():
        for i,(x,y) in enumerate(test_loader):
            try:
                label = np.zeros((x.shape[0], 10))
                label[np.arange(x.shape[0]), y] = 1
                label = torch.tensor(label)

                pred, mu, logvar = model(x.to(device),label.to(device))
                recon_loss, kld = loss_function(x.to(device),pred, mu, logvar)
                loss = recon_loss + kld

                total_loss += loss.cpu().data.numpy()*x.shape[0]
                reconstruction_loss += recon_loss.cpu().data.numpy()*x.shape[0]
                kld_loss += kld.cpu().data.numpy()*x.shape[0]
                if i == 0:
                    # print("gr:", x[0,0,:5,:5])
                    # print("pred:", pred[0,0,:5,:5])
                    plot(epoch, pred.cpu().data.numpy(), y.cpu().data.numpy())
            except Exception as e:
                #traceback.print_exe()
                torch.cuda.empty_cache()
                continue
    reconstruction_loss /= len(test_loader.dataset)
    kld_loss /= len(test_loader.dataset)
    total_loss /= len(test_loader.dataset)
    return total_loss, kld_loss,reconstruction_loss        

def load_data():
    """transform = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()])
    train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('./data/', train=True, download=True,
                             transform=transform),batch_size=batch_size, num_workers=num_workers, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('./data/', train=False, download=True,
                             transform=transform),batch_size=batch_size, num_workers=num_workers, shuffle=True)
    """
    #dataset = pd.read_csv('./../accordion345.csv')
    #dataset = pd.read_csv('./../Violin.csv')
    dataset = pd.read_csv('./../Strings.csv')
    #dataset = pd.read_csv('./../TinySOL_metadata_updated.csv')

    dataset['Full_Path'] = './../TinySOL/' + dataset['Path'].astype(str)
    data = dataset[['Full_Path','Instance ID']]
    data.head()
    
    myds = SoundDS(data, './',
               duration=1,
               sr=44100,
               #n_fft=2048,
               n_fft=8192,
               hop_len=512,
               label='Instance ID',
               cutoff=128,
               highpass=False
               )

    #batch_size = 32
    batch_size = 1


    # Random split of 80:20 between training and validation
    num_items = len(myds)
    num_train = round(num_items * 0.8)
    num_val = num_items - num_train
    train_ds, val_ds = random_split(myds, [num_train, num_val])

    # Create training and validation data loaders
    train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, num_workers=num_workers, shuffle=False)

    return train_dl, val_dl


def save_model(model, epoch):
    if not os.path.isdir("./checkpoints"):
        os.mkdir("./checkpoints")
    file_name = './checkpoints/model_{}.pt'.format(epoch)
    torch.save(model.state_dict(), file_name)

    

In [ ]:

# dont use for now

# L1 works like a charm
#criterion = nn.L1Loss()

log_interval = 10

lr = 0.0001



def train_es(epochs, model, train_dl, val_dl, patience = 10):

    # Writer will output to ./runs/ directory by default
    logdir = './runs/l2_kld_h100_es_ ' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    writer = SummaryWriter(logdir)
    
    log_interval = 10
    lr = 0.0001

    optimizer = Adam(model.parameters(), lr=lr)
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    class_size=4

    print(device)

    for epoch in range(1, epochs + 1):

        model.train()
        train_loss = 0

        for batch_idx, (spectrogram,label) in enumerate(train_dl):

            #scipy works with cpu when applying the highpass filter
            spectrogram_tensor = spectrogram.to(dtype=torch.float32)
            #print("spectrogram tensor:",spectrogram_tensor.shape)

            label = label.to(device)
            data = spectrogram_tensor.to(device)
            label = one_hot(label, class_size) # class number, not sure yet 

            optimizer.zero_grad()

            recon_batch, mu, logvar = model(data.to(device),label.to(device))
            
            # recon_batch is prediction, data is ground_truth        
            #criterion is for l1 or mse
            #loss = criterion(recon_batch, data.view(-1, x_dim))

            #print('recon batch:',recon_batch.shape)

            loss = loss_function(recon_batch, data, mu, logvar)

            loss.backward()

            train_loss += loss.item()
            writer.add_scalar('Loss/train', train_loss, epoch)
            optimizer.step()
            
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.4f}'.format(
                    epoch, batch_idx * len(data), len(train_dl.dataset),
                    100. * batch_idx / len(train_dl), loss.item()))

        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(train_dl.dataset)))


        """if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_dl.dataset),
                    100. * batch_idx / len(train_dl),
                    loss.item() / len(data)))

        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(train_dl.dataset)))"""

        # Validation
        model.eval()
        val_loss = 0

        with torch.no_grad():

            for i, (spectrogram,labels) in enumerate(val_dl):
                float32_tensor = spectrogram.to(dtype=torch.float32)
                data, labels = float32_tensor.to(device),labels.to(device)
                labels = one_hot(labels, class_size) # 

                recon_batch, mu, logvar = model(data,labels)
                
                #val_loss += criterion(recon_batch, data[0]) #.item()
                val_loss += loss_function(recon_batch, data, mu, logvar).item()

                #val_loss = criterion(recon_batch, data.view(-1, x_dim))
                
                #spec = recon_batch.squeeze().cpu().numpy()

                #AudioUtil.plot_spectrogram(spec)

                #x_rec, rmse = reconstruct_signal_griffin_lim(mag_subset.T, 2048, 512, 30)
                #x_rec, rmse = reconstruct_signal_griffin_lim(spec.T, 2048, 512, 30)            

                #sf.write(f'./reconstructed_violin_high_pass/test_reconst_{i}.wav', x_rec, 44100)
                
                #sf.write(f'./reconstructed_accordion/test_reconst_{i}.wav', x_rec, 44100)

        #val_loss /= len(val_dl.dataset)

        # Calculate average losses
        train_loss /= len(train_dl.dataset)
        val_loss /= len(val_dl.dataset)
        
        print('====> Test set loss: {:.4f}'.format(val_loss))

        early_stopping(val_loss)

        if early_stopping.early_stop:
            print("Early stopping")
            print("Saving Model")
            save_model(model, epoch)
            break

    writer.close()



In [ ]:
train_loader, test_loader = load_data()
print("dataloader created")
input_dim=4097*87
#input_dim=1025*87
hidden_dim=400
latent_size=50
num_classes=4
model = Model(input_dim=input_dim, hidden_dim=hidden_dim, latent_size=latent_size,num_classes=num_classes).to(device)
print("model created")


for batch_idx, (spectrogram,label) in enumerate(train_loader):

            #scipy works with cpu when applying the highpass filter
    #spectrogram_tensor = spectrogram.to(dtype=torch.float32)
    print(spectrogram.shape)
    break


In [ ]:
patience = 3
#patience = 10
#epochs = 100
epochs=100
train_es(epochs, model, train_dl=train_loader, val_dl=test_loader, patience=patience)
#train(epochs)


In [ ]:
data = []
data_mu = []
data_logvar = []
targets = []

n_samples = int(len(train_loader))
counter = 0

class_size=4
#x_dim = 1025*259
x_dim = 4097*87

for spectrogram, labels in tqdm(train_loader):
#for batch_idx, (spectrogram, labels) in enumerate(train_dl):

  float32_tensor = spectrogram.to(dtype=torch.float32)
  spec_data, labels_enc = float32_tensor.to(device),labels.to(device)
  labels_enc = one_hot(labels_enc, class_size) # 

  #spec_data = spec_data.view(-1, x_dim)

  mu, logvar = model.encode(spec_data,labels_enc)

  z = model.reparameterize(mu, logvar).detach().cpu().numpy()
  
  for x, y in zip(z, labels):
    data.append(x)
    targets.append(y)
    counter += 1

  if counter >= n_samples:
    break

data = np.array(data)
targets = np.array(targets)

df = pd.DataFrame({"x":data[:, 0], "y":data[:,1], "hue":targets})
sns.scatterplot(x="x", y="y", hue="hue", data=df, legend='full')

In [ ]:
#save_model(model, 27)

In [ ]:
def test(epoch,val_dl):
    model.eval()
    test_loss = 0

    with torch.no_grad():
        for i, (spectrogram, labels) in enumerate(val_dl):
            float32_tensor = spectrogram.to(dtype=torch.float32)
            data, labels = float32_tensor.to(device),labels.to(device)
            labels = one_hot(labels, class_size) # 

            recon_batch, mu, logvar = model(data, labels)
            print('recon_batch:',recon_batch.shape)
            #print('label:',labels[0])

            #test_loss += criterion(recon_batch, data) #.item()
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

            #if i==0:
                #print(spec.shape)
                #1025, 431
            spec = recon_batch.squeeze().cpu().numpy()
            print('spec prev reshape:',spec.shape)
            #spec = recon_batch.view(batch_size, 1, 1025, 431).squeeze().cpu().numpy()
            
            #spec = recon_batch.view(batch_size, 1, 1025, 259).squeeze().cpu().numpy()
            
            spec = recon_batch.view(batch_size, 1, 4097, 87).squeeze().cpu().numpy()

            print('spec reshape:',spec.shape)
            print('spec to plot:',spec.shape)
            AudioUtil.plot_spectrogram(spec)
            print(spec[-1].shape)

            #x_rec, rmse = reconstruct_signal_griffin_lim(mag_subset.T, 2048, 512, 30)
            4097, 87
            #x_rec, rmse = reconstruct_signal_griffin_lim(spec.T, 2048, 512, 30)            

            #x_rec, rmse = reconstruct_signal_griffin_lim(spec.T, 8192, 512, 30)            
            
            x_rec, rmse = reconstruct_signal_griffin_lim(spec.T, 8192, 512, 30)            


            #sf.write(f'./reconstructed_accordion/test_reconst_{i}.wav', x_rec, 44100)
            sf.write(f'./reconstructed/test_reconst_{i}.wav', x_rec, 44100)

    test_loss /= len(val_dl.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


epochs = 15
class_size=4
for epoch in range(1, epochs + 1):
    #c = torch.eye(431,7).cuda()
    test(epoch,test_loader)

In [ ]:
instances = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],[1,0,1,0]]

for instance in instances:
    with torch.no_grad():

        c = torch.tensor(instance).to(device)

        num_samples = 1  # Number of samples to generate
        
        # Sample from latent space
        #sample = torch.randn(num_samples, latent_size).to(device)
        z = torch.randn(num_samples, latent_size).to(device)
        # Decode samples
        recon_batch = model.decode(z, c).cpu() 

        print(recon_batch.shape)
        #torch.Size([1, 1025, 259])
    #    spec = recon_batch.view(batch_size, 1, 1025, 431).squeeze().cpu().numpy()

        spec = recon_batch.view(batch_size, 1, 4097, 87).squeeze().cpu().numpy()

        #spec = recon_batch.view(batch_size, 1, 1025, 259).squeeze().cpu().numpy()
        
        print(instance)
        print(spec.shape)

        #noisy
        AudioUtil.plot_spectrogram(spec)

        #x_rec, rmse = reconstruct_signal_griffin_lim(spec.T, 2048, 512, 30)            
        x_rec, rmse = reconstruct_signal_griffin_lim(spec.T, 8192, 512, 30)            
        
        sf.write(f'./generated/test_gen_{str(instance)}.wav', x_rec, 44100)

        
        
        # Estimate the noise spectrum

    """
        threshold = 0.15
        spec[spec < threshold] = threshold

        AudioUtil.plot_spectrogram(spec)


        x_rec, rmse = reconstruct_signal_griffin_lim(spec.T, 2048, 512, 30)            

        sf.write(f'./generated_27/test_gen_fixed_denoised.wav', x_rec, 44100)
    """



if load_epoch > 0:
    model.load_state_dict(torch.load('./checkpoints/model_{}.pt'.format(load_epoch), map_location=torch.device('mps')))
    print("model {} loaded".format(load_epoch))

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)


train_loss_list = []
test_loss_list = []
for i in range(load_epoch+1, max_epoch):
    model.train()
    train_total, train_kld, train_loss = train(i, model, train_loader, optimizer)
    #with torch.no_grad():
    #    model.eval()
    #    test_total, test_kld, test_loss = test(i, model, test_loader)
    #    if generate:
    #        z = torch.randn(6, 32).to(device)
    #        y = torch.tensor([1,2,3,4,5,6]) - 1
    #        generate_image(i,z, y, model)
        
    print("Epoch: {}/{} Train loss: {}, Train KLD: {}, Train Reconstruction Loss:{}".format(i, max_epoch,train_total, train_kld, train_loss))
    #print("Epoch: {}/{} Test loss: {}, Test KLD: {}, Test Reconstruction Loss:{}".format(i, max_epoch, test_loss, test_kld, test_loss))

    save_model(model, i)
    train_loss_list.append([train_total, train_kld, train_loss])
    #test_loss_list.append([test_total, test_kld, test_loss])
    np.save("train_loss", np.array(train_loss_list))
    #np.save("test_loss", np.array(test_loss_list))


# i, (example_data, exaple_target) = next(enumerate(test_loader))
# print(example_data[0,0].shape)
# plt.figure(figsize=(5,5), dpi=100)
# plt.imsave("example.jpg", example_data[0,0], cmap='gray',  dpi=1000)